# Modélisation du réseau

### Library

In [257]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json 
import seaborn as sns
import utm

import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression

from utils import *

import os

### Parameters

In [182]:
tension_level = ['400kV', '225kV','90kV', '63kV', '45kV', '<45kV']
tension_layer = 0

## Datas 

### Lignes

1. Aériennes

In [ ]:
path_ligne_aerienne = '/Users/matthiasmolenat/repos/congestion/data/2412_lignes-aeriennes-rte-nv.csv'
df_ligne_RT_a= pd.read_csv(path_ligne_aerienne,delimiter=';')

2. Souterraines

In [184]:
path_ligne_souterraine = "/Users/matthiasmolenat/repos/congestion/data/2412_lignes-souterraines-rte-nv.csv"
df_ligne_RT_s= pd.read_csv(path_ligne_souterraine,delimiter=";")

3. Concatennation

In [185]:
df_ligne_RT = pd.concat([df_ligne_RT_a,df_ligne_RT_s])

In [186]:
# calcul de la colonne des longueur
if True :

    geo_shape_index = df_ligne_RT.columns.to_list().index("Geo Shape")

    for index, row in df_ligne_RT.iterrows():
        geo_data = json.loads(row['Geo Shape'])
        longitudes, latitudes = zip(*geo_data["coordinates"])
        df_ligne_RT.at[index, "Length (m)"] = length_from_coordinate(longitudes, latitudes)

In [ ]:
df_ligne_RT.shape

## Postes

In [188]:
path_postes= '/Users/matthiasmolenat/repos/congestion/data/2412_postes-electriques-rte.csv'
df_postes_RT= pd.read_csv(path_postes,delimiter=';')

## Flux de puissance

### Consommation des métropôles

Consommation des métrôpoles françaises : https://odre.opendatasoft.com/explore/dataset/eco2mix-metropoles-tr/information/?disjunctive.libelle_metropole&disjunctive.nature

In [189]:
path_metropoles= '/Users/matthiasmolenat/repos/congestion/data/eco2mix-metropoles-tr.csv'
df_metropoles= pd.read_csv(path_metropoles,delimiter=';')

In [190]:
df_metropoles_grouped = df_metropoles.groupby(['Métropole'])
df_metropoles_separated = {group_name[0]: sub_df for group_name, sub_df in df_metropoles_grouped}

In [ ]:
df_metropoles_separated['Bordeaux Métropole']

Contour simplifié des EPCI : https://www.data.gouv.fr/fr/datasets/referent-contours-des-epci-2023/

In [314]:
path_EPCI= '/Users/matthiasmolenat/repos/congestion/data/referents-contours-epci.csv'
df_EPCI= pd.read_csv(path_EPCI,delimiter=';')
df_EPCI_metro = df_EPCI[df_EPCI['code_epci'].isin(df_metropoles['Code métropole'].unique())]

### Profil de consommation par section d'activité

https://odre.opendatasoft.com/explore/dataset/soutirages-quotidiens-consolides-rpt-par-section/export/?disjunctive.libelle_tension&disjunctive.secteur_activite_economique&refine.date_de_la_courbe=2023

In [193]:
path_profil_secteurs= '/Users/matthiasmolenat/repos/congestion/data/soutirages-quotidiens-consolides-rpt-par-section.csv'
df_profil_secteurs= pd.read_csv(path_profil_secteurs,delimiter=';')

In [194]:
df_profil_secteurs_grouped = df_profil_secteurs.groupby(['Secteur activité'])
df_profil_secteurs_separated = {group_name[0]: sub_df for group_name, sub_df in df_profil_secteurs_grouped}

In [ ]:
print(df_profil_secteurs['Secteur activité'].unique())

### Cosommation annuelle par EPCI

https://data.enedis.fr/explore/dataset/consommation-electrique-par-secteur-dactivite-epci/table/?sort=conso_moyenne_mwh&refine.annee=2023

In [291]:
path_conso_annuel_EPCI= '/Users/matthiasmolenat/repos/congestion/data/consommation-electrique-par-secteur-dactivite-epci.csv'
df_conso_annuel_secteurs= pd.read_csv(path_conso_annuel_EPCI,delimiter=';')

col_to_drop =["Conso moyenne (MWh)", "Nombre de mailles secretisées",
       "Part thermosensible (%)", "Conso totale  usages thermosensibles (MWh)",
       "Conso totale  usages non thermosensibles (MWh)",
       "Thermosensibilité totale (kWh/DJU)",
       "Conso totale corrigée de l'aléa climatique  usages thermosensibles (MWh)",
       "Conso moyenne usages thermosensibles (MWh)",
       "Conso moyenne  usages non thermosensibles (MWh)",
       "Thermosensibilité moyenne (kWh/DJU)",
       "Conso moyenne corrigée de l'aléa climatique  usages thermosensibles (MWh)",
       "DJU à TR", "DJU à TN", "Geom"]
df_conso_annuel_secteurs.drop(col_to_drop, axis=1, inplace=True)

index_to_drop = df_conso_annuel_secteurs.index[~df_conso_annuel_secteurs['Code EPCI'].str.isdigit()]
df_conso_annuel_secteurs.drop(index_to_drop, axis=0, inplace=True)



In [ ]:
print(df_conso_annuel_secteurs['CODE GRAND SECTEUR'].unique())
print(df_conso_annuel_secteurs['Nom Région'].unique())

### Registre national des installations de production et de stockage d'électricité

https://odre.opendatasoft.com/explore/dataset/registre-national-installation-production-stockage-electricite-agrege/export/?disjunctive.departement&disjunctive.region&disjunctive.epci&disjunctive.filiere&disjunctive.technologie&disjunctive.combustiblessecondaires&disjunctive.combustible&disjunctive.gestionnaire&disjunctive.regime&sort=combustible

In [286]:
path_puissance_installee = '/Users/matthiasmolenat/repos/congestion/data/registre-national-installation-production-stockage-electricite-agrege.csv'
df_puissance_installee= pd.read_csv(path_puissance_installee,delimiter=';', low_memory=False)
col_to_drop =['idPEPS', 'nomInstallation', 'codeEICResourceObject', 'codeIRIS',
       'codeINSEECommune', 'commune', 'EPCI', 'codeDepartement',
       'departement', 'codeRegion', 'codeIRISCommuneImplantation', 'energieAnnuelleGlissanteSoutiree',
       'codeINSEECommuneImplantation', 'codeS3RENR', 'dateRaccordement',
       'dateDeraccordement', 'dateMiseEnService', 'dateDebutVersion',
       'tensionRaccordement', 'modeRaccordement','filiere', 'codeFiliere', 'codeCombustible', 'combustible',
       'codesCombustiblesSecondaires', 'combustiblesSecondaires', 'posteSource',
       'codeTechnologie', 'typeStockage', 'puisMaxInstallee', 'energieAnnuelleGlissanteProduite',
       'puisMaxRacCharge', 'puisMaxCharge', 'puisMaxRac', 'energieAnnuelleGlissanteStockee',
       'puisMaxInstalleeDisCharge', 'nbGroupes', 'nbInstallations', 'regime',
       'energieStockable', 'capaciteReservoir', 'hauteurChute', 'productible',
       'debitMaximal', 'codeGestionnaire', 'gestionnaire', 'dateMiseEnservice (format date)']
df_puissance_installee.drop(col_to_drop, axis=1, inplace=True)
line_to_drop = df_puissance_installee[(df_puissance_installee['region'] == 'Guadeloupe') |
                                                         (df_puissance_installee['region'] == 'La Réunion') |
                                                         (df_puissance_installee['region'] == 'Guyane') |
                                                         (df_puissance_installee['region'] == 'Corse') |
                                                         (df_puissance_installee['region'] == 'Martinique')].index
df_puissance_installee.drop(line_to_drop, axis=0, inplace=True)
df_puissance_installee.dropna(subset=['technologie', 'codeEPCI'], inplace=True)
df_puissance_installee['codeEPCI'] = df_puissance_installee['codeEPCI'].astype(int)
df_puissance_installee = df_puissance_installee.groupby(['technologie','region', 'codeEPCI']).sum()

In [287]:
df_puissance_installee.head()

energieAnnuelleGlissanteInjectee  \
technologie region               codeEPCI                                      
Autre       Auvergne-Rhône-Alpes 200034882                        24869906.0   
                                 200040491                         1472519.0   
                                 200040681                         5216854.0   
                                 200040715                        12898943.0   
                                 200040798                           11151.0   

                                            maxPuis  
technologie region               codeEPCI            
Autre       Auvergne-Rhône-Alpes 200034882   5250.0  
                                 200040491    195.0  
                                 200040681   2152.0  
                                 200040715   3600.0  
                                 200040798   4000.0

### Injections régionales quotidiennes consolidées sur le réseau de transport d'électricité (2019 à 2024)

In [ ]:
path_prod_region = '/Users/matthiasmolenat/repos/congestion/data/injections-regionales-quotidiennes-consolidees-rpt.csv'
df_prod_region= pd.read_csv(path_prod_region,delimiter=';')
df_prod_region.head()

## Specific extraction 

Based on selected layer of tension

In [271]:
selected_tension = tension_level[:tension_layer+1]
df_postes_RT = df_postes_RT[df_postes_RT['Tension (kV)'].isin(selected_tension)]
df_ligne_RT = df_ligne_RT[df_ligne_RT['TENSION'].isin(selected_tension)]
df_ligne_RT = df_ligne_RT[df_ligne_RT['Length (m)']>30]
df_ligne_RT.reset_index(inplace=True)


In [ ]:
df_postes_RT.head()

## Exploitation

Network as a graph

Surpiquage missing

In [ ]:
from scipy.sparse import csr_matrix

# Extract the coordinates of the extremities
coordinates = []
for index, row in df_ligne_RT.iterrows():
    geo_data = json.loads(row['Geo Shape'])
    coordinates.append(geo_data["coordinates"][0])  # Start point
    coordinates.append(geo_data["coordinates"][-1])  # End point

# Calculate the distance matrix
dist_matrix = distance_matrix_geodesic(coordinates)

# Create a sparse matrix with edges where distance is less than or equal to 200 meters
threshold = 30  # distance in coordinates
adjacency_matrix = (dist_matrix <= threshold).astype(int)

# Convert to sparse matrix
sparse_matrix = csr_matrix(adjacency_matrix)
sparse_matrix.setdiag(0)
print(sparse_matrix)



Compare injection and consumption by EPCI

In [315]:
df_conso_per_EPCI = df_conso_annuel_secteurs.groupby(['Code EPCI']).sum()
df_conso_per_EPCI.index = df_conso_per_EPCI.index.astype(int)
df_conso_per_EPCI = df_conso_per_EPCI['Conso totale (MWh)']


df_prod_per_EPCI = df_puissance_installee.groupby(['codeEPCI']).sum()
df_prod_per_EPCI.index = df_prod_per_EPCI.index.astype(int)
df_prod_per_EPCI = df_prod_per_EPCI['energieAnnuelleGlissanteInjectee']

df_compared = pd.merge(df_conso_per_EPCI, df_prod_per_EPCI, right_index= True, left_index= True, how='inner')
df_compared = pd.merge(df_EPCI[['geo_shape','code_epci']], df_compared, right_index= True, left_on= 'code_epci', how='inner')
df_compared['Injection totale (MWh)'] = df_compared['energieAnnuelleGlissanteInjectee']/1000
df_compared.drop(['energieAnnuelleGlissanteInjectee'], axis=1, inplace=True)
df_compared['ratio prod/conso %'] = df_compared['Injection totale (MWh)']/df_compared['Conso totale (MWh)']*100


In [316]:
df_compared.head()

,geo_shape,code_epci,Conso totale (MWh),Injection totale (MWh),ratio prod/conso %
0,"{""coordinates"": [[[0.604566, 45.694384], [0.60...",200072049,220739.167,206915.907,93.737740
1,"{""coordinates"": [[[2.311766, 43.115752], [2.29...",200035715,543237.324,144961.473,26.684741
2,"{""coordinates"": [[[2.097452, 43.964379], [2.08...",200040905,59230.955,83066.811,140.242228
3,"{""coordinates"": [[[5.68684, 45.484412], [5.672...",200068567,359376.797,15580.193,4.335336
4,"{""coordinates"": [[[1.879758, 48.440855], [1.86...",200073344,400276.733,49293.255,12.314794


In [319]:
df_compared['ratio prod/conso %'].quantile([.1, 0.25,0.5,0.75, .9])

0.10      2.207254
0.25      7.255577
0.50     20.807657
0.75     69.969697
0.90    294.978074
Name: ratio prod/conso %, dtype: float64

## Exploitation 

- Définir des scénarios de puissance types : 2 journées (week-end / jour férié + milieu de semaine) pour chaque mois de l'année, pour un total de 24 scénario à faire tourner.
- Associer les puissances de produciton et de consommation aux postes considérables les plus proches. Utiliser les centroïdes.
- Faire un écoulement de puissance optimal pour déterminer la dynamique des flux dans le réseau.

## Visualisation

Visualisation des lignes

In [ ]:
show_map_lines(df_ligne_RT, True)


Visualisation de EPCI avec ratio production